[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/appautomaton/code-cli-tunnel/blob/main/code-cli.ipynb)

# code-cli tunnel

This notebook prepares your remote Compute Instance with code-cli tunneling capability.


# [IGNORE] Use `Code-CLI` if you like to do a real dev-tunnel

> Tunnel ONLY -> only open up one particular port using devtunnel

`!curl -sL https://aka.ms/DevTunnelCliInstall | bash`
```python
!/root/bin/devtunnel create colab
!/root/bin/devtunnel port create -p 2222 --protocol http
!/root/bin/devtunnel host colab
```

# Setup `code-cli`
## [Colab ONLY] patch the config file

In [ ]:
from pathlib import Path

target = Path("/etc/ipython/ipython_config.py")
backup = target.with_suffix(target.suffix + ".bak")  # /etc/ipython/ipython_config.py.bak

if not target.exists():
    raise FileNotFoundError(f"No system IPython config at {target}")

text = target.read_text(encoding="utf-8")

if "google.colab._kernel.Kernel" not in text:
    print("No Colab kernel_class setting found; skipping.")
elif "When running outside the Colab Python environment" in text:
    print("Already patched; nothing to do.")
else:
    if not backup.exists():
        backup.write_text(text, encoding="utf-8")
        print(f"Backed up to: {backup}")

    old_kernel = "c.IPKernelApp.kernel_class = 'google.colab._kernel.Kernel'\n"
    new_kernel = (
        "try:\n"
        "  import google.colab._kernel  # noqa: F401\n"
        "except Exception:\n"
        "  # When running outside the Colab Python environment (e.g. a micromamba env),\n"
        "  # the google.colab package may not be installed. Keep defaults in that case.\n"
        "  pass\n"
        "else:\n"
        "  c.IPKernelApp.kernel_class = 'google.colab._kernel.Kernel'\n"
    )

    old_ext = (
        "c.InteractiveShellApp.extensions = [\n"
        "    'google.colab',\n"
        "]\n"
    )
    new_ext = (
        "try:\n"
        "  import google.colab  # noqa: F401\n"
        "except Exception:\n"
        "  c.InteractiveShellApp.extensions = []\n"
        "else:\n"
        "  c.InteractiveShellApp.extensions = [\n"
        "      'google.colab',\n"
        "  ]\n"
    )

    updated = text.replace(old_kernel, new_kernel).replace(old_ext, new_ext)

    if updated == text:
        print("No exact matches found; nothing changed (your config may differ).")
    else:
        target.write_text(updated, encoding="utf-8")
        print(f"Patched: {target}\nRestart the runtime/kernel for it to take effect.")



## Install code-cli

In [ ]:
%%bash
set -e
cd /content

ARCH="$(uname -m)"
if [ "$ARCH" = "x86_64" ]; then OS="cli-alpine-x64";
elif [ "$ARCH" = "aarch64" ] || [ "$ARCH" = "arm64" ]; then OS="cli-alpine-arm64";
else echo "Unsupported arch: $ARCH"; exit 1; fi

wget -O code.tgz "https://code.visualstudio.com/sha/download?build=stable&os=${OS}"
tar -xzf code.tgz
chmod +x ./code

## Open the `tunnel` named `colab`
> change the `--name` to anyhing desired

In [ ]:
!./code tunnel user login --provider github
!./code tunnel --name colab --accept-server-license-terms